# ReSurfEMG: a Python package for comprehensive analysis of respiratory surface EMG


### Note
This notebook is adapted to run on ReSurfEMG 1.1.0 to improve forward compatability and re-use. As compared to ReSurfEMG 1.0.2, with which the analysis for the paper was conducted, this mainly includes refactoring of methods and no signal analytical adapations:
- Signal indexing: TimeSeries.y_raw -->   TimeSeries['raw]
- Convert signal_type to signal_io arguments: signal_type='raw' -->   signal_io=('raw', 'filt')
- Split gating and wavelet_denoising in ECG peak detection (.get_ecg_peaks) and ECG removal (.gating, .wavelet_denoising)

# 1. Import the required libraries

In [ ]:
# Standard code libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Custom code libraries from ReSurfEMG
from resurfemg.data_connector.config import Config
from resurfemg.data_connector import file_discovery
from resurfemg.pipelines import ipy_widgets
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader
from resurfemg.data_connector.data_classes import (
VentilatorDataGroup, EmgDataGroup)
from resurfemg.postprocessing import features as feat

config = Config()
%matplotlib widget

In [ ]:
output_dir = os.path.join(config.get_directory('output_data'), 'software_paper')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
plt.rcParams['svg.fonttype'] = 'none'


## 2. Load the ventilator and sEMG data

In [ ]:
# Identify all recordings available for the selected patient/measurement_date

# First find the patients
base_path = config.get_directory('patient_data')
patient_paths = file_discovery.find_folders(
    base_path,
    folder_levels=['patient'])
patient = list(patient_paths['patient'].values)[0]

# Then find the files for the selected patients:
folder_levels = ['date', 'measurement']
patient_path = os.path.join(base_path, patient)

emg_files = file_discovery.find_files(
    base_path=patient_path,
    file_name_regex='EMG_recording',
    extension_regex='poly5',
    folder_levels=folder_levels)

folder_levels = ['date', 'measurement']
vent_files = file_discovery.find_files(
    base_path=patient_path,
    file_name_regex='Draeger_recording',
    extension_regex='poly5',
    folder_levels=folder_levels)

button_list = ipy_widgets.file_select(
    emg_files,
    folder_levels=folder_levels[:-1],
    default_value_select=[None, '006'],
    default_idx_select=[0, 6])

In [ ]:
emg_file_chosen = os.path.join(
    patient_path, *[btn.value for btn in button_list], 'EMG_recording.Poly5')
vent_file_chosen = os.path.join(
    patient_path, *[btn.value for btn in button_list], 'Draeger_recording.Poly5')

print("The chosen files are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
fs_vent = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg = data_emg_samples[:, fs_emg:]
y_vent = data_vent_samples[:, fs_vent:]

# Define the time axes
t_emg = [i/fs_emg for i in range(len(y_emg[0, :]))]
t_vent = [i/fs_vent for i in range(len(y_vent[0, :]))]

In [ ]:
# Store the EMG data in a group of TimeSeries objects
emg_timeseries = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi', 'EMGpara'],
    units=3*['uV'])

# Store the ventilator data in a group of TimeSeries objects
vent_timeseries = VentilatorDataGroup(
    y_vent,
    fs=fs_vent,
    labels=['Paw', 'F', 'Vvent'],
    units=['cmH2O', 'L/s', 'L'])


In [ ]:
x_lim = 8.0
y_lim = 10.0

In [ ]:
vent_timeseries[0].baseline(
    signal_io=('raw', 'baseline'),
)

# Find tidal volume peaks
p_vent = vent_timeseries[vent_timeseries.p_vent_idx]
v_vent = vent_timeseries[vent_timeseries.v_vent_idx]
vent_timeseries.find_tidal_volume_peaks(
    start_idx=0,
    overwrite=True
)
p_vent.peaks['ventilator_breaths'].detect_on_offset(
    baseline=p_vent['baseline'])
v_vent.peaks['ventilator_breaths'].detect_on_offset(
    baseline=v_vent['baseline'])

# 3a. Under-smoothed

In [ ]:
emg_timeseries_under = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi', 'EMGpara'],
    units=3*['uV'])

emg_timeseries_under.run(
    'filter_emg',
    order=1,
    hp_cf=15,
    lp_cf=1000,
)
emg_timeseries_under.run('get_ecg_peaks', overwrite=True)
emg_timeseries_under.run(
    'gating',
    fill_method=3,
    gate_width_samples=(150 *  fs_emg) // 1000
)
emg_timeseries_under.run(
    'envelope',
    env_type='rms',
    env_window=(100 * fs_emg) // 1000,
    ci_alpha=0.05,
)
emg_timeseries_under.run(
    'baseline',
    percentile=20,
    window_s=int(7.5 * fs_emg)
)
emg_di_under = emg_timeseries_under.channels[1]
emg_di_under.detect_emg_breaths(
    peak_set_name='breaths',
    start_idx=0,
    min_peak_width_s=fs_emg//10,
    prominence_factor=0.5,
    overwrite=True
)
emg_di_under.peaks['breaths'].detect_on_offset(
    baseline=emg_di_under['baseline']
)
sEAdi_amplitudes = feat.amplitude(
    signal=emg_di_under['env'],
    peak_idxs=emg_di_under.peaks['breaths']['peak_idx'],
    baseline=emg_di_under['baseline'],
)
sEAdi_ttp_fs, _ = feat.time_to_peak(
    emg_env=emg_di_under['env'],
    start_idxs=emg_di_under.peaks['breaths']['start_idx'],
    end_idxs=emg_di_under.peaks['breaths']['end_idx'],
)
sEAdi_ttp = sEAdi_ttp_fs / emg_di_under.param['fs']

sEAdi_pseudo_slope_fs = feat.pseudo_slope(
    emg_env=emg_di_under['env'],
    start_idxs=emg_di_under.peaks['breaths']['start_idx'],
    end_idxs=emg_di_under.peaks['breaths']['end_idx'],
    smoothing=False
)
sEAdi_pseudo_slope = sEAdi_pseudo_slope_fs * emg_di_under.param['fs']

emg_di_under.peaks['breaths'].peak_df['amplitude'] = sEAdi_amplitudes
emg_di_under.peaks['breaths'].peak_df['time_to_peak'] = sEAdi_ttp
emg_di_under.peaks['breaths'].peak_df['pseudo_slope'] = sEAdi_pseudo_slope

# emg_di_under.peaks['breaths'].peak_df
len(emg_di_under.peaks['breaths'].peak_df)

# 3b. Default settings

In [ ]:
emg_timeseries_good = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi', 'EMGpara'],
    units=3*['uV'])

emg_timeseries_good.run(
    'filter_emg',
    order=3,
    hp_cf=20,
    lp_cf=500,
)

emg_timeseries_good.run('get_ecg_peaks', overwrite=True)
emg_timeseries_good.run(
    'gating',
    fill_method=3,
    gate_width_samples=(200 *  fs_emg) // 1000
)
emg_timeseries_good.run(
    'envelope',
    env_type='rms',
    env_window=(250 * fs_emg) // 1000,
    ci_alpha=0.05,
)
emg_timeseries_good.run(
    'baseline',
    percentile=33,
    window_s=int(7.5 * fs_emg)
)
emg_di_good = emg_timeseries_good[1]
emg_di_good.detect_emg_breaths(
    peak_set_name='breaths',
    start_idx=0,
    min_peak_width_s=fs_emg//4,
    prominence_factor=1.0,
)
emg_di_good.peaks['breaths'].detect_on_offset(
    baseline=emg_di_good['baseline']
)

sEAdi_amplitudes = feat.amplitude(
    signal=emg_di_good['env'],
    peak_idxs=emg_di_good.peaks['breaths']['peak_idx'],
    baseline=emg_di_good['baseline'],
)
sEAdi_ttp_fs, _ = feat.time_to_peak(
    emg_env=emg_di_good['env'],
    start_idxs=emg_di_good.peaks['breaths']['start_idx'],
    end_idxs=emg_di_good.peaks['breaths']['end_idx'],
)
sEAdi_ttp = sEAdi_ttp_fs / emg_di_good.param['fs']

sEAdi_pseudo_slope_fs = feat.pseudo_slope(
    emg_env=emg_di_good['env'],
    start_idxs=emg_di_good.peaks['breaths']['start_idx'],
    end_idxs=emg_di_good.peaks['breaths']['end_idx'],
    smoothing=False
)
sEAdi_pseudo_slope = sEAdi_pseudo_slope_fs * emg_di_good.param['fs']

emg_di_good.peaks['breaths'].peak_df['amplitude'] = sEAdi_amplitudes
emg_di_good.peaks['breaths'].peak_df['time_to_peak'] = sEAdi_ttp
emg_di_good.peaks['breaths'].peak_df['pseudo_slope'] = sEAdi_pseudo_slope

len(emg_di_good.peaks['breaths'].peak_df)

# 3c. Over-smoothed

In [ ]:
emg_timeseries_over = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi', 'EMGpara'],
    units=3*['uV'])

emg_timeseries_over.run(
    'filter_emg',
    order=3,
    hp_cf=30,
    lp_cf=500,
)
emg_timeseries_over.run('get_ecg_peaks', overwrite=True)
emg_timeseries_over.run(
    'gating',
    fill_method=3,
    gate_width_samples=(300 *  fs_emg) // 1000,
)
emg_timeseries_over.run(
    'envelope',
    env_type='rms',
    env_window=(500 * fs_emg) // 1000,
    ci_alpha=0.05,
)
emg_timeseries_over.run(
    'baseline',
    percentile=33,
    window_s=int(7.5 * fs_emg)
)
emg_di_over = emg_timeseries_over[1]
emg_di_over.detect_emg_breaths(
    peak_set_name='breaths',
    start_idx=0,
    min_peak_width_s=fs_emg//4,
    prominence_factor=0.25,
)
emg_di_over.peaks['breaths'].detect_on_offset(
    baseline=emg_di_over['baseline']
)

sEAdi_amplitudes = feat.amplitude(
    signal=emg_di_over['env'],
    peak_idxs=emg_di_over.peaks['breaths']['peak_idx'],
    baseline=emg_di_over['baseline'],
)
sEAdi_ttp_fs, _ = feat.time_to_peak(
    emg_env=emg_di_over['env'],
    start_idxs=emg_di_over.peaks['breaths']['start_idx'],
    end_idxs=emg_di_over.peaks['breaths']['end_idx'],
)
sEAdi_ttp = sEAdi_ttp_fs / emg_di_over.param['fs']

sEAdi_pseudo_slope_fs = feat.pseudo_slope(
    emg_env=emg_di_over['env'],
    start_idxs=emg_di_over.peaks['breaths']['start_idx'],
    end_idxs=emg_di_over.peaks['breaths']['end_idx'],
    smoothing=False
)
sEAdi_pseudo_slope = sEAdi_pseudo_slope_fs * emg_di_over.param['fs']

emg_di_over.peaks['breaths'].peak_df['amplitude'] = sEAdi_amplitudes
emg_di_over.peaks['breaths'].peak_df['time_to_peak'] = sEAdi_ttp
emg_di_over.peaks['breaths'].peak_df['pseudo_slope'] = sEAdi_pseudo_slope


len(emg_di_over.peaks['breaths'].peak_df)

# 4. Calculate features

In [ ]:
# Calculate ETPdi & Test EMG quality
parameter_names = {
    'time_product': 'ETPdi'
}
# Under-filtered pipeline
emg_di_under.calculate_time_products(
    peak_set_name='breaths', parameter_name='ETPdi')
emg_di_under.calculate_time_products(
    peak_set_name='breaths', include_aub=False, parameter_name='ETPdi-aub')
emg_di_under.test_emg_quality(
    peak_set_name='breaths', parameter_names=parameter_names, verbose=False)


# Good pipeline
emg_di_good.calculate_time_products(
    peak_set_name='breaths', parameter_name='ETPdi')
emg_di_good.calculate_time_products(
    peak_set_name='breaths', include_aub=False, parameter_name='ETPdi-aub')
emg_di_good.test_emg_quality(
    peak_set_name='breaths', parameter_names=parameter_names, verbose=False)

# Over-filtered pipeline
emg_di_over.calculate_time_products(
    peak_set_name='breaths', parameter_name='ETPdi')
emg_di_over.calculate_time_products(
    peak_set_name='breaths', include_aub=False, parameter_name='ETPdi-aub')
emg_di_over.test_emg_quality(
    peak_set_name='breaths', parameter_names=parameter_names, verbose=False)



In [ ]:
# Link the breaths to the ventilator breaths
t_vent_breaths = \
    p_vent.peaks['ventilator_breaths']['peak_idx'] / p_vent.param['fs']
t_vent_breaths = t_vent_breaths - 0.5

emg_di_under.link_peak_set(
    peak_set_name='breaths',
    t_reference_peaks=t_vent_breaths,
    linked_peak_set_name='linked_breaths',
)
emg_di_good.link_peak_set(
    peak_set_name='breaths',
    t_reference_peaks=t_vent_breaths,
    linked_peak_set_name='linked_breaths',
)
emg_di_over.link_peak_set(
    peak_set_name='breaths',
    t_reference_peaks=t_vent_breaths,
    linked_peak_set_name='linked_breaths',
)

# 4a. Full recording metrics

In [ ]:
emg_di_under.peaks['linked_breaths'].peak_df['T_breath'] = (
    (emg_di_under.peaks['linked_breaths'].peak_df['end_idx']
     - (emg_di_under.peaks['linked_breaths'].peak_df['start_idx']))
     / emg_di_under.param['fs'])

under_output = pd.concat(
    [emg_di_under.peaks['linked_breaths'].peak_df.loc[:, ['T_breath', 'time_to_peak', 'amplitude', 'ETPdi', 'pseudo_slope']],
     emg_di_under.peaks['linked_breaths'].quality_values_df[['snr', 'aub', 'bell']]],
    axis=1
)
under_output.describe()

In [ ]:
emg_di_good.peaks['linked_breaths'].peak_df['T_breath'] = (
    (emg_di_good.peaks['linked_breaths'].peak_df['end_idx']
     - (emg_di_good.peaks['linked_breaths'].peak_df['start_idx']))
     / emg_di_good.param['fs'])

good_output = pd.concat(
    [emg_di_good.peaks['linked_breaths'].peak_df.loc[:, ['T_breath', 'time_to_peak', 'amplitude', 'ETPdi', 'pseudo_slope']],
     emg_di_good.peaks['linked_breaths'].quality_values_df[['snr', 'aub', 'bell']]],
    axis=1
)
good_output.describe()

In [ ]:
emg_di_over.peaks['linked_breaths'].peak_df['T_breath'] = (
    (emg_di_over.peaks['linked_breaths'].peak_df['end_idx']
     - (emg_di_over.peaks['linked_breaths'].peak_df['start_idx']))
     / emg_di_over.param['fs'])

over_output = pd.concat(
    [emg_di_over.peaks['linked_breaths'].peak_df.loc[:, ['T_breath', 'time_to_peak', 'amplitude', 'ETPdi', 'pseudo_slope']],
     emg_di_over.peaks['linked_breaths'].quality_values_df[['snr', 'aub', 'bell']]],
    axis=1
)
over_output.describe()

## 4b. Three peak features

In [ ]:
under_output.loc[:2, :]

In [ ]:
good_output.loc[:2, :]

In [ ]:
over_output.loc[:2, :]

# 5. Plot selected peaks

In [ ]:
# Plot the identified Pocc peaks in p_vent and sEAdi
fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(10, 6), sharex=True)
axes_emg = axis[:, 0]
colors = ['tab:cyan', 'tab:orange', 'tab:red']
emg_timeseries_under[1].plot_full(axes_emg[0], signal_io=('env',), colors=['tab:red', 'tab:orange'])
emg_di_under.plot_markers(peak_set_name='linked_breaths', axes=axes_emg[0], colors=['k', 'k', 'k'])
axes_emg[0].set_ylabel('sEAdi (uV)')
axes_emg[0].set_ylim([0, y_lim])

emg_timeseries_good[1].plot_full(axes_emg[1], signal_io=('env',), colors=['tab:green', 'tab:olive'])
emg_di_good.plot_markers(peak_set_name='linked_breaths', axes=axes_emg[1], colors=['k', 'k', 'k'])
axes_emg[1].set_ylabel('sEAdi (uV)')
axes_emg[1].set_ylim([0, y_lim])

emg_timeseries_over[1].plot_full(axes_emg[2], signal_io=('env',), colors=['tab:blue', 'tab:cyan'])
emg_di_over.plot_markers(peak_set_name='linked_breaths', axes=axes_emg[2], colors=['k', 'k', 'k'])
axes_emg[2].set_ylabel('sEAdi (uV)')
axes_emg[2].set_ylim([0, y_lim])

axes_emg[0].set_title('EMG data')
axes_emg[-1].set_xlabel('t (s)')

axes_vent = axis[:, 1]
vent_timeseries.run('plot_full', axes=axes_vent)

axes_vent[0].set_title('Ventilator data')
axes_vent[-1].set_xlabel('t (s)')
axes_vent[-1].set_xlim([0, x_lim])
axes_emg[1].set_ylim([0, y_lim])

In [ ]:
fig_path = os.path.join(output_dir, 'three_pipelines.svg')
# fig.savefig(fig_path, format='svg', bbox_inches='tight')